In [43]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "D:/ML/"
print(DOWNLOAD_ROOT)
DATASET_PATH = os.path.join("datasets", "Countries")
DIR_PATH = {"Afganistan":os.path.join(DATASET_PATH, "Afganistan"),
            "China":os.path.join(DATASET_PATH, "China"),
            "CAR":os.path.join(DATASET_PATH, "CAR"),
           "Ethiopia":os.path.join(DATASET_PATH, "Ethiopia"),
           "India":os.path.join(DATASET_PATH, "India"),
           "Iran":os.path.join(DATASET_PATH, "Iran"),
            "KNDR":os.path.join(DATASET_PATH, "KNDR"),
            "Libya":os.path.join(DATASET_PATH, "Libya"),
           "Morocco":os.path.join(DATASET_PATH, "Morocco"),
           "Sudan":os.path.join(DATASET_PATH, "Sudan"),
           "Syria":os.path.join(DATASET_PATH, "Syria"),
           "Turkey":os.path.join(DATASET_PATH, "Turkey"),
           }
filenames = {}
for items in DIR_PATH.keys():
    print(items)
    filenames[items] = [name for name in sorted(os.listdir(DIR_PATH[items])) if len(name) > 20] 

D:/ML/
Afganistan
China
CAR
Ethiopia
India
Iran
KNDR
Libya
Morocco
Sudan
Syria
Turkey


In [44]:
from charset_normalizer import from_path
import string

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content

import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
spec_chars = string.punctuation + '\r' + '\n\xa0«»\t—…' 

x_temp = []
y_temp = []
for item in filenames.keys():
    post_temp=[]
    for names in filenames[item]:
        content = str(from_path(DIR_PATH[item]+"\\"+names).best()).lower()  
        content = remove_emojis(content)
        # content = re.sub(r'(\\u[0-9A-Fa-f]+)', lambda matchobj: chr(int(matchobj.group(0)[2:], 16)), content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_temp.append(content)
        y_temp.append(item)

In [45]:
import numpy as np
from sklearn.model_selection import train_test_split


X = np.array(x_temp, dtype=object)
y = np.array(y_temp)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [46]:
try:
    import urlextract # may require an Internet connection to download root domain names
    
    url_extractor = urlextract.URLExtract()
except ImportError:
    url_extractor = None

In [47]:
from sklearn.base import BaseEstimator, TransformerMixin
import urlextract # may require an Internet connection to download root domain names
import re
from collections import Counter
import nltk
from tokenizer_exceptions import normalizer_exc_rus
ru_stopwords = set(
        """
    а авось ага агу аж ай али алло ау ах ая
    б будем будет будете будешь буду будут будучи будь будьте бы был была были было
    быть бац без безусловно бишь благо благодаря ближайшие близко более больше
    будто бывает бывала бывали бываю бывают бытует
    в вам вами вас весь во вот все всё всего всей всем всём всеми всему всех всею
    всея всю вся вы ваш ваша ваше ваши вдали вдобавок вдруг ведь везде вернее
    взаимно взаправду видно вишь включая вместо внакладе вначале вне вниз внизу
    вновь вовсе возможно воистину вокруг вон вообще вопреки вперекор вплоть
    вполне вправду вправе впрочем впрямь вресноту вроде вряд всегда всюду
    всякий всякого всякой всячески вчеред
    г го где гораздо гав
    д да для до дабы давайте давно давным даже далее далеко дальше данная
    данного данное данной данном данному данные данный данных дану данунах
    даром де действительно довольно доколе доколь долго должен должна
    должно должны должный дополнительно другая другие другим другими
    других другое другой
    е его едим едят ее её ей ел ела ем ему емъ если ест есть ешь еще ещё ею едва
    ежели еле
    ж же
    з за затем зато зачем здесь значит зря
    и из или им ими имъ их ибо иль имеет имел имела имело именно иметь иначе
    иногда иным иными итак ишь
    й
    к как кем ко когда кого ком кому комья которая которого которое которой котором
    которому которою которую которые который которым которыми которых кто ка кабы
    каждая каждое каждые каждый кажется казалась казались казалось казался казаться
    какая какие каким какими каков какого какой какому какою касательно кой коли
    коль конечно короче кроме кстати ку куда
    л ли либо лишь любая любого любое любой любом любую любыми любых
    м меня мне мной мною мог моги могите могла могли могло могу могут мое моё моего
    моей моем моём моему моею можем может можете можешь мои мой моим моими моих
    мочь мою моя мы мало меж между менее меньше мимо многие много многого многое
    многом многому можно мол му
    н на нам нами нас наса наш наша наше нашего нашей нашем нашему нашею наши нашим
    нашими наших нашу не него нее неё ней нем нём нему нет нею ним ними них но
    наверняка наверху навряд навыворот над надо назад наиболее наизворот
    наизнанку наипаче накануне наконец наоборот наперед наперекор наподобие
    например напротив напрямую насилу настоящая настоящее настоящие настоящий
    насчет нате находиться начала начале неважно негде недавно недалеко незачем
    некем некогда некому некоторая некоторые некоторый некоторых некто некуда
    нельзя немногие немногим немного необходимо необходимости необходимые
    необходимым неоткуда непрерывно нередко несколько нету неужели нечего
    нечем нечему нечто нешто нибудь нигде ниже низко никак никакой никем
    никогда никого никому никто никуда ниоткуда нипочем ничего ничем ничему
    ничто ну нужная нужно нужного нужные нужный нужных ныне нынешнее нынешней
    нынешних нынче
    о об один одна одни одним одними одних одно одного одной одном одному одною
    одну он она оне они оно от оба общую обычно ого однажды однако ой около оный
    оп опять особенно особо особую особые откуда отнелижа отнелиже отовсюду
    отсюда оттого оттот оттуда отчего отчему ох очевидно очень ом
    п по при паче перед под подавно поди подобная подобно подобного подобные
    подобный подобным подобных поелику пожалуй пожалуйста позже поистине
    пока покамест поколе поколь покуда покудова помимо понеже поприще пор
    пора посему поскольку после посреди посредством потом потому потомушта
    похожем почему почти поэтому прежде притом причем про просто прочего
    прочее прочему прочими проще прям пусть
    р ради разве ранее рано раньше рядом
    с сам сама сами самим самими самих само самого самом самому саму свое своё
    своего своей своем своём своему своею свои свой своим своими своих свою своя
    себе себя собой собою самая самое самой самый самых сверх свыше се сего сей
    сейчас сие сих сквозь сколько скорее скоро следует слишком смогут сможет
    сначала снова со собственно совсем сперва спокону спустя сразу среди сродни
    стал стала стали стало стать суть сызнова
    та то ту ты ти так такая такие таким такими таких такого такое такой таком такому такою
    такую те тебе тебя тем теми тех тобой тобою того той только том томах тому
    тот тою также таки таков такова там твои твоим твоих твой твоя твоё
    теперь тогда тоже тотчас точно туда тут тьфу тая
    у уже увы уж ура ух ую
    ф фу
    х ха хе хорошо хотел хотела хотелось хотеть хоть хотя хочешь хочу хуже
    ч чего чем чём чему что чтобы часто чаще чей через чтоб чуть чхать чьим
    чьих чьё чё
    ш ша
    щ ща щас
    ы ых ые ый
    э эта эти этим этими этих это этого этой этом этому этот этою эту эдак эдакий
    эй эка экий этак этакий эх
    ю
    я явно явных яко якобы якоже
    
    и что не это  быть этот это свой как - этот весь быть что ▫ но г. %
    """.split()
    )
# stemmer = nltk.PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")
def Make_dict(X_text, strip_headers=True, lower_case=True,
    remove_punctuation=True, replace_urls=True, replace_numbers=True,
    stemming=True, remove_stopwords=True):
    
    X_transformed = []
    for email in X_text:
        text = email or ""
        text = normalizer_exc_rus(text)
        if lower_case:
            text = text.lower()
        if replace_urls and url_extractor is not None:
            urls = list(set(url_extractor.find_urls(text)))
            urls.sort(key=lambda url: len(url), reverse=True)
            for url in urls:
                text = text.replace(url, "")
        if replace_numbers:
            text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', '', text)
        if remove_punctuation:
            text = re.sub(r'\W+', ' ', text, flags=re.M)
        if remove_stopwords:
            filtered_words = []
            for token in text.split():
                if token not in ru_stopwords and len(token)>2:
                    filtered_words.append(token)
    
            # Join the filtered words to form a clean text
            text = ' '.join(filtered_words)
        word_counts = Counter(text.split())
        if stemming and stemmer is not None:
            stemmed_word_counts = Counter()
            for word, count in word_counts.items():
                stemmed_word = stemmer.stem(word)
                stemmed_word_counts[stemmed_word] += count
            word_counts = stemmed_word_counts
        X_transformed.append(word_counts)
    return np.array(X_transformed)

In [48]:
X_train_dic = Make_dict(X_train, stemming=False)
X_train_words = []
for post in X_train_dic:
    for word in post.keys():
        X_train_words.append(word)
print(X_train_words [:7])


['распп', 'приглашает', 'предпринимателей', 'сферы', 'креативных', 'индустрий', 'принять']


In [49]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(vocabulary = set(X_train_words))
X_train_transformed = count.fit_transform(X_train)

In [50]:
X_train_transformed.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [51]:
X_train_transformed.shape

(4195, 64099)

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver="lbfgs", max_iter=5000, random_state=42, n_jobs=-1)
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=3, verbose=3)
score.mean()

[CV] END ................................ score: (test=0.894) total time=   1.8s
[CV] END ................................ score: (test=0.896) total time=   1.7s
[CV] END ................................ score: (test=0.888) total time=   1.7s


0.8927290867548624

In [53]:
count.get_feature_names_out()

array(['aaa', 'aamiddleeast', 'aap', ..., 'ፖሊስ', 'ፖሊቴክኒክ', '፲፪ቱ'],
      dtype=object)

In [54]:
from sklearn.metrics import precision_score, recall_score

X_test_transformed = count.transform(X_test)

log_clf = LogisticRegression(solver="lbfgs", max_iter=6000, random_state=42)
log_clf.fit(X_test_transformed, y_test)

y_pred = log_clf.predict(X_test_transformed)

print("Precision: {:.2f}%".format(100 * precision_score(y_test, y_pred, average='micro')))
print("Recall: {:.2f}%".format(100 * recall_score(y_test, y_pred, average='micro')))

Precision: 99.43%
Recall: 99.43%


In [65]:
import pickle
with open('contry_post-model.bin', 'wb') as f_out:
    pickle.dump([log_clf, X_train_words], f_out)

In [55]:
new_text = ['Нигер последовал примеру Мали, разорвав связи с Украиной после того, как пресс-секретарь разведывательного агентства Министерства обороны Украины признал, что Украина оказывала поддержку повстанцам, которые убили малийских вооруженных сил и российских агентов 25-27 июля в Тинзауатене, на севере Мали, недалеко от границы с Алжиром.Посол Украины в Сенегале также выразил безоговорочную поддержку малийским повстанцам, что привело к вызову посла в МИД Сенегала.',
            'Еще новости афганской промышленности. В Кандагаре спустя 18 лет возобновила работу текстильная фабрика. Официальные лица говорят, что они отремонтировали оборудование на фабрике за шесть месяцев, сообщают афганское СМИ. По словам работников фабрики, с возобновлением работы фабрики созданы рабочие места. 69-летний Мохаммад, проработавший на текстильной фабрике в Кандагаре значительное время, рад вернуться на работу.',
           'Накопленное непонимание между Китаем и США не может быть решено лишь одним подобным изящным жестом. Страны расходятся по широкому кругу вопросов, санкционный механизм против Китая не ослаблен. Поэтому отправка панд – это скорее демонстрация миролюбивой позиции Китая в противовес США.',
            'В Алжире, как правило, на похоронах мужчины сидят около дома, хорошо, если есть сад или терраса, а женщины в доме, надеть платок женщина должна обязательно, даже если в обычной жизни она не покрывает голову. Если покойный жил в квартире, то прямо во дворе ставят стулья и шатры со столами для мужчин.Готовят либо нанятые кухарки, либо родственницы покойного. Пока составляла пост от подруги услышала,что в их семье часто еду приносят те, кто приходит в дом, а готовить должны невестки, а не дочери умершего. Мой муж сказал, что это совершенно необязательно, на похоронах его бабушки готовили только её дочери, это было их желание и никаких особых правил на счёт этого нет. Обязательного блюда на поминки в Алжире нет. Народ приходит помянутьот трех дней до недели, если у человека была большая семья и много знакомых!Кормят традиционными блюдами: суп-шорба, кус-кус/беркукес/тлитли с мясом и т.д.Могилу посещают каждое утро, в течении трех дней. Через 40 дней устанавливают небольшое надгробие, никаких памятников,вычурных элементов. На надгробии имя, фамилия, даты рождения и смерти, фотографий нет. Вроде бы написала всё, что знала. Задавайте вопросы в комментариях, если таковые имеются.',
            'Сегодня в Венесуэле проходят президентские выборы. Мой коллега Дмитрий Морозов рассуждает, как их итоги могут повлиять на дальнейший путь страны. Вот самое важное из его статьи.📍В выборах один тур, участвуют десять кандидатов, однако только двое имеют реальные шансы на победу: действующий президент Николас Мадуро и Эдмундо Гонсалес Уррутия, кандидат от Единой демократической платформы, объединяющей наиболее значимые оппозиционные партии.'
           ]
x_temp = []
for post_temp in new_text:
        content = str(post_temp).lower()  
        content = remove_emojis(content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_temp.append(content)
   

count = CountVectorizer(vocabulary = set(X_train_words))
X_t = count.transform(x_temp)
accuracy = log_clf.predict(X_t)
print (accuracy)
print ('Predicted Probabilities: {} - {}'.format(log_clf.predict(X_t), log_clf.predict_proba(X_t)))

['CAR' 'Afganistan' 'China' 'India' 'China']
Predicted Probabilities: ['CAR' 'Afganistan' 'China' 'India' 'China'] - [[0.0448222  0.36177531 0.11550723 0.04667658 0.11084489 0.08282343
  0.0305585  0.02454119 0.04100519 0.05247193 0.02360906 0.06536449]
 [0.44815044 0.06460173 0.14934964 0.05469975 0.11821455 0.06541259
  0.0077006  0.01619594 0.01583552 0.02617368 0.00706328 0.02660227]
 [0.01251511 0.03820323 0.74257934 0.01038435 0.13160822 0.02950085
  0.00261869 0.00527948 0.0049725  0.0089878  0.0051484  0.00820203]
 [0.03275996 0.06204704 0.0857326  0.06225232 0.49906975 0.14011849
  0.00654088 0.01455936 0.02650484 0.03389317 0.00807525 0.02844632]
 [0.03130359 0.12612958 0.4384318  0.02873554 0.16295036 0.10619044
  0.00893254 0.01639647 0.01606741 0.01604553 0.00837593 0.0404408 ]]


In [56]:
DOWNLOAD_ROOT = "D:/ML/"
print(DOWNLOAD_ROOT)
DATASET_PATH_test = os.path.join("datasets", "spam")
DIR_PATH_test = {"Africa":os.path.join(DATASET_PATH_test, "Africa"),
            "China":os.path.join(DATASET_PATH_test, "China"),
            "MiddleEAST":os.path.join(DATASET_PATH_test, "MiddleEAST"),
           "LatAmerica":os.path.join(DATASET_PATH_test, "LatAmerica"),
           "IranPakistanAfganistan":os.path.join(DATASET_PATH_test, "IranPakistanAfganistan"),}
filenames = {}
for items in DIR_PATH_test.keys():
    print(items)
    filenames[items] = [name for name in sorted(os.listdir(DIR_PATH_test[items])) if len(name) > 20] 

D:/ML/
Africa
China
MiddleEAST
LatAmerica
IranPakistanAfganistan


In [57]:
from charset_normalizer import from_path
import string

def remove_chars_from_text(text, chars):
    # return "".join([ch for ch in text if ch not in chars])
    content = ''
    for ch in text:
        if ch not in chars:
            content = content + ''.join(ch)
        else:
            content = content + ''.join(' ')
    return content

import re

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
spec_chars = string.punctuation + '\r' + '\n\xa0«»\t—…' 

x_test = []
for item in filenames.keys():
    post_temp=[]
    for names in filenames[item]:
        content = str(from_path(DIR_PATH_test[item]+"\\"+names).best()).lower()  
        content = remove_emojis(content)
        # content = re.sub(r'(\\u[0-9A-Fa-f]+)', lambda matchobj: chr(int(matchobj.group(0)[2:], 16)), content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_test.append(content)       

In [58]:
len(x_test)

6906

In [62]:
import pandas as pd

x_temp = []
for post_temp in x_test[-150:-100]:
        content = str(post_temp).lower()  
        content = remove_emojis(content)
        content = remove_chars_from_text(content, spec_chars)
        content = remove_chars_from_text(content, string.digits)
        for i in range(1, 10):
            content = content.replace('  ', ' ')
        x_temp.append(content)
count = CountVectorizer(vocabulary = set(X_train_words))
X_t = count.transform(x_temp)
accuracy = log_clf.predict(X_t)

print (accuracy)
df = pd.DataFrame(log_clf.predict_proba(X_t), columns=log_clf.classes_)
df_max = pd.DataFrame({'text': x_temp,'predict': log_clf.predict(X_t), 'accuracy': df.max(axis=1)})
df_max


['Afganistan' 'Afganistan' 'Afganistan' 'China' 'Afganistan' 'Afganistan'
 'Afganistan' 'China' 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan'
 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan'
 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan'
 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan'
 'Afganistan' 'Afganistan' 'Afganistan' 'China' 'China' 'Afganistan'
 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan'
 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan' 'Afganistan'
 'Afganistan' 'Afganistan' 'Iran' 'Afganistan' 'Afganistan' 'Afganistan'
 'Afganistan']


text     predict  accuracy
0   проведение новой международной встречи спецпре...  Afganistan  0.971464
1   flydubai возобновила полеты в афганистан после...  Afganistan  0.973447
2   руководство афганистана не в полной мере контр...  Afganistan  0.996161
3   разведчик й роты го ооспн гру кандагар младший...       China  0.381928
4   сегодня в москве проходит международная конфер...  Afganistan  0.973647
5   сегодня в москве проходит международная конфер...  Afganistan  0.973647
6    афганскому народу нужно политическое решениек...  Afganistan  0.781917
7   на украине сша наступят на вьетнамские афганск...       China  0.561345
8    никто в талибане не готов к компромиссу а нек...  Afganistan  0.974867
9   находящиеся в пакистане иностранные иммигранты...  Afganistan  0.994815
10  запад хочет сделать украину для россии вторым ...  Afganistan  0.684881
11  сша самый главный нарушитель прав человека в а...  Afganistan  0.930344
12  афганистан больше не лидер по производству опи...  Afganistan  0.935723
13  афганистан удвоил закупки спг из россииафганис...  Afganistan  0.752765
14  талибы не обеспечили безопасность соседних стр...  Afganistan  0.998827
15  представители национальных общин нескольких ре...  Afganistan  0.966866
16  дорогие подписчики друзья спешу поздравить вас...  Afganistan  0.305054
17  россия доставила в четверг в кабул очередную п...  Afganistan  0.994606
18  россия доставила в четверг в кабул очередную п...  Afganistan  0.994606
19  украину ждет судьба афганистана ирака ливии и ...  Afganistan  0.782148
20  в афганистане уже есть инклюзивное и ответстве...  Afganistan  0.990863
21  интересно конечно как некоторые каналы выдают ...  Afganistan  0.591763
22  выжившие после авиакатастрофы в афганистане ро...  Afganistan  0.941663
23  имеющие влияние на фнса силы тоже должны поощр...  Afganistan  0.966574
24  талибы выступили за укрепление регионального э...  Afganistan  0.999707
25  россия на предстоящей встрече в дохе по афгани...  Afganistan  0.981022
26  организация договора о коллективной безопаснос...  Afganistan  0.913111
27  в москве сочли провальной вторую конференцию п...  Afganistan  0.884870
28  глава комитета палаты представителей по иностр...  Afganistan  0.994563
29  численность боевиков исламского государства на...  Afganistan  0.789812
30  знакомьтесь с дашей российским журналистом афр...       China  0.661739
31  знакомьтесь с дашей российским журналистом афр...       China  0.661739
32  главной ошибкой соединенных штатов было то что...  Afganistan  0.997547
33  отец погибшего в афганистане американского вое...  Afganistan  0.388629
34  организация договора о коллективной безопаснос...  Afganistan  0.795087
35  интересно афганские нелегалы в великобритании ...  Afganistan  0.751876
36  я раньше много писала про женский бизнес в афг...  Afganistan  0.952493
37  госдепартамент сша слишком поздно принял решен...  Afganistan  0.928605
38  спецслужбы казахстана пресекли канал незаконно...  Afganistan  0.958545
39  талибы осудили теракт в крокус сити холле мид ...  Afganistan  0.991415
40  теракты свидетествуют о слабости врагов россии...  Afganistan  0.973888
41  фаузия куфи бывший вице спикер афганского парл...  Afganistan  0.437764
42  спецпредставитель президента ирана по афганист...  Afganistan  0.748405
43  в последние годы игил осуществляет вылазки про...  Afganistan  0.939179
44  министерство промышленности и торговли талибов...  Afganistan  0.960137
45  если бы не сотрудничество ирана и россии в сир...        Iran  0.624705
46  борьба с террористическими группировками в афг...  Afganistan  0.993098
47  сейчас центры игил находятся за пределами афга...  Afganistan  0.997358
48  дорогие подписчики нам срочно надо поговорить ...  Afganistan  0.627948
49  иг не располагает возможностями для вербовки н...  Afganistan  0.831991